In [1]:
import tensorflow as tf
import numpy as np

In [2]:
data = np.loadtxt('./data.csv', delimiter=',',
                  unpack=True, dtype='float32')

In [3]:
x_data = np.transpose(data[0:2])
y_data = np.transpose(data[2:])
print('x_data :', x_data)
print('y_data :', y_data)

x_data : [[ 0.  0.]
 [ 1.  0.]
 [ 1.  1.]
 [ 0.  0.]
 [ 0.  0.]
 [ 0.  1.]]
y_data : [[ 1.  0.  0.]
 [ 0.  1.  0.]
 [ 0.  0.  1.]
 [ 1.  0.  0.]
 [ 1.  0.  0.]
 [ 0.  0.  1.]]


In [4]:
# 학습에 직접적으로 사용하지 않고 학습 횟수에 따라 단순히 증가시킬 변수를 만듭니다.
global_step = tf.Variable(0, trainable=False, name='global_step')
print(global_step)

<tf.Variable 'global_step:0' shape=() dtype=int32_ref>


In [5]:
X = tf.placeholder(tf.float32)
Y = tf.placeholder(tf.float32)

W1 = tf.Variable(tf.random_uniform([2, 10], -1., 1.))
L1 = tf.nn.relu(tf.matmul(X, W1))

W2 = tf.Variable(tf.random_uniform([10, 20], -1., 1.))
L2 = tf.nn.relu(tf.matmul(L1, W2))

W3 = tf.Variable(tf.random_uniform([20, 3], -1., 1.))
model = tf.matmul(L2, W3)

In [6]:
cost = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=model))

optimizer = tf.train.AdamOptimizer(learning_rate=0.01)
# global_step로 넘겨준 변수를, 학습용 변수들을 최적화 할 때 마다 학습 횟수를 하나씩 증가시킵니다.
train_op = optimizer.minimize(cost, global_step=global_step)

In [7]:
sess = tf.Session()
# global_variables 함수를 통해 앞서 정의하였던 변수들을 저장하거나 불러올 변수들로 설정합니다.
saver = tf.train.Saver(tf.global_variables())

In [8]:
ckpt = tf.train.get_checkpoint_state('./model')
if ckpt and tf.train.checkpoint_exists(ckpt.model_checkpoint_path):
    saver.restore(sess, ckpt.model_checkpoint_path)
else:
    sess.run(tf.global_variables_initializer())

In [9]:
for step in range(100):
    sess.run(train_op, feed_dict={X: x_data, Y: y_data})
    
    print('Step: %d, ' % sess.run(global_step),
          'Cost: %.3f' % sess.run(cost, feed_dict={X: x_data, Y: y_data}))

Step: 1,  Cost: 1.062
Step: 2,  Cost: 1.020
Step: 3,  Cost: 0.984
Step: 4,  Cost: 0.952
Step: 5,  Cost: 0.921
Step: 6,  Cost: 0.893
Step: 7,  Cost: 0.867
Step: 8,  Cost: 0.842
Step: 9,  Cost: 0.820
Step: 10,  Cost: 0.799
Step: 11,  Cost: 0.780
Step: 12,  Cost: 0.762
Step: 13,  Cost: 0.746
Step: 14,  Cost: 0.731
Step: 15,  Cost: 0.716
Step: 16,  Cost: 0.702
Step: 17,  Cost: 0.689
Step: 18,  Cost: 0.676
Step: 19,  Cost: 0.665
Step: 20,  Cost: 0.653
Step: 21,  Cost: 0.643
Step: 22,  Cost: 0.633
Step: 23,  Cost: 0.624
Step: 24,  Cost: 0.616
Step: 25,  Cost: 0.608
Step: 26,  Cost: 0.601
Step: 27,  Cost: 0.596
Step: 28,  Cost: 0.591
Step: 29,  Cost: 0.586
Step: 30,  Cost: 0.582
Step: 31,  Cost: 0.579
Step: 32,  Cost: 0.575
Step: 33,  Cost: 0.572
Step: 34,  Cost: 0.570
Step: 35,  Cost: 0.567
Step: 36,  Cost: 0.565
Step: 37,  Cost: 0.564
Step: 38,  Cost: 0.562
Step: 39,  Cost: 0.561
Step: 40,  Cost: 0.559
Step: 41,  Cost: 0.558
Step: 42,  Cost: 0.557
Step: 43,  Cost: 0.557
Step: 44,  Cost: 0.5

In [10]:
saver.save(sess, './model/dnn.ckpt', global_step=global_step)

'./model/dnn.ckpt-100'

In [11]:
#########
# 결과 확인
# 0: 기타 1: 포유류, 2: 조류
######
prediction = tf.argmax(model, 1)
target = tf.argmax(Y, 1)
print('예측값:', sess.run(prediction, feed_dict={X: x_data}))
print('실제값:', sess.run(target, feed_dict={Y: y_data}))

예측값: [0 1 2 0 0 2]
실제값: [0 1 2 0 0 2]


In [12]:
check_prediction = tf.equal(prediction, target)
accuracy = tf.reduce_mean(tf.cast(check_prediction, tf.float32))
print('정확도: %.2f' % sess.run(accuracy * 100, feed_dict={X: x_data, Y: y_data}))

정확도: 100.00
